# 진행 과정 정리해보기

영화 시놉시스 코퍼스로 영화 구분과 영화 장르 간에 내재된 편향성을 측정해보자.

1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출한다.
2. 추출된 결과로 embedding model을 만든다.
3. target, attribute 단어 셋을 만든다.
4. 모든 장르에 대해 WEAT score를 계산하고 이를 heatmap으로 만든다.


# 실험 계획 세우기

TF-IDF를 이용해서 Target과 attribute 단어 셋을 만드는 것이 노드의 방식이었다.

하지만, 빈도 기반의 방식이기 때문에 단어의 의미를 벡터로 표현하지 못한다는 한계가 있다.

물론 전처리를 통해서 TF-IDF를 개선해서 실험해보는 방법도 있겠으나, 영화 특성 상 의미나 주제가 중요할 것이므로 LSA와 LDA 방식으로 실험해보기로 했다.

시간 관계상 우선 LSA를 먼저 구현해보고, 시간이 되면 LDA를 구현해야겠다.



## 데이터 다운로드

In [2]:
# 시놉시스 데이터 다운로드
#!unzip ~/work/weat/data/synopsis.zip -d ~/work/weat/data/

In [2]:
# 경로 설정
import os

data_dir = os.path.join(os.getenv("HOME"), "work/weat/data")
file_name = os.path.join(data_dir, "synopsis.txt")

with open(file_name, 'r') as file:
    for i in range(20):
        print(file.readline(), end='')

사운드 엔지니어 상우(유지태 분)는 치매에 걸린 할머니(백성희 분)와
 젊은 시절 상처한 한 아버지(박인환 분), 고모(신신애 분)와 함께 살고 있다.
 어느 겨울 그는 지방 방송국 라디오 PD 은수(이영애 분)를 만난다.
 자연의 소리를 채집해 틀어주는 라디오 프로그램을 준비하는 은수는 상우와 녹음 여행을 떠난다.
 자연스레 가까워지는 두 사람은 어느 날, 은수의 아파트에서 밤을 보낸다.
 너무 쉽게 사랑에 빠진 두 사람... 상우는 주체할 수 없을 정도로 그녀에게 빨려든다.
 그러나 겨울에 만난 두 사람의 관계는 봄을 지나 여름을 맞이하면서 삐걱거린다.
 이혼 경험이 있는 은수는 상우에게 결혼할 생각이 없다며 부담스러운 표정을 내비친다.
 "어떻게 사랑이 변하니?..."라고 묻는 상우에게 은수는 그저 "헤어져" 라고 단호하게 말한다.
 영원히 변할 것 같지 않던 사랑이 변하고, 그 사실을 받아들이지 못하는 상우는 어찌 할 바를 모른다.
 은수를 잊지 못하는 상우는 미련과 집착의 감정을 이기지 못하고 서울과 강릉을 오간다.
유사 이래 연령, 성별, 빈부의 차이와 정치적인 입장을 불문하고 일거에 국민을 통합해 온 '애국심'이라는 성역에 일침을 가하는 다큐멘터리. 재작년 전국 민족민주 유가족협의회의 장기농성을 다룬 인상적인 다큐멘터리 <민들레>를 만들었던 독립영화집단 '빨간 눈사람'이 우리 사회 구석구석을 발빠르게 돌아다니며 애국심과 민족주의가 강요되는 현장을 발굴하여 카메라에 담았다. 박홍 서강대 명예총장, 이도형 '한국논단' 발행인, 축구해설자 신문선, 홍세화, 박노해 등 사회 각계의 '스타'들이 등장해 저마다의 확고한 신념을 성토한다. 감독 이경순과 최하동하는 이 작품을 위해 3년간 백여 명을 인터뷰했다고 한다. 2001 올해의 독립영화상 수상.
 민족과 국가란 공동체에서 부단히 권력과 부를 얻는 자, 나아가 민족과 국가란 공동체에서 얻은 신분과 부귀를 영원히 그의 자손에게 대물림하려는 자, 그래서 민족과 국가란 공동체를 부단히 유지해야만 하는 자, 따라서

## 형태소 분석기로 품사가 명사인 경우 해당 단어 추출하기

In [3]:
# Konlpy 형태소 분석기 다운로드
!pip install konlpy

In [4]:
#======품사가 명사인 경우 해당 단어 추출하기==========
from konlpy.tag import Okt

okt = Okt()
tokenized = []

with open(file_name, 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        words = okt.pos(line, stem=True, norm=True)
        res = []
        for w in words:
            if w[1] in ["Noun"]:      # "Adjective", "Verb" 등을 포함할 수도 있습니다.
                res.append(w[0])    # 명사일 때만 tokenized 에 저장하게 됩니다.
        tokenized.append(res)

print("품사가 명사인 경우 해당 단어 추출함")

품사가 명사인 경우 해당 단어 추출함


In [7]:
# 명사가 모여있는 tokenized 길이
print(len(tokenized))

71156


## 추출된 결과로 embedding vector 만들기

In [8]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)
#tokenized를 CBOW 방식으로 임베딩 벡터를 만들었다.


# 테스트하기
model.wv.most_similar(positive=['영화'])

[('작품', 0.8932034373283386),
 ('다큐멘터리', 0.8523848056793213),
 ('드라마', 0.8311743140220642),
 ('영화로', 0.8013336062431335),
 ('감동', 0.7956811785697937),
 ('소재', 0.7919965386390686),
 ('코미디', 0.7895067930221558),
 ('형식', 0.7873854637145996),
 ('스토리', 0.7800410389900208),
 ('개연', 0.7763791680335999)]

# Target, Attribute 단어 셋 만들기

## 먼저 TF-IDF 만들기

영화 종류 별 대표 단어 추출하기

In [9]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'

def read_token(file_name):
    okt = Okt()
    result = []
    with open(data_dir + '/' + file_name, 'r') as fread:
        print(file_name, '파일을 읽고 있습니다.')
        while True:
            line = fread.readline()
            if not line: break
            tokenlist = okt.pos(line, stem=True, norm=True)
            for word in tokenlist:
                if word[1] in ["Noun"]:#, "Adjective", "Verb"]:
                    result.append((word[0]))
    return ' '.join(result)

print("슝~")

슝~


In [10]:
# 영화 장르를 토큰화
art = read_token(art_txt)
gen = read_token(gen_txt)

synopsis_art.txt 파일을 읽고 있습니다.
synopsis_gen.txt 파일을 읽고 있습니다.


In [11]:
# 타겟 X
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([art, gen])

print(X.shape)

(2, 41082)


In [12]:
print(vectorizer.vocabulary_['영화'])
print(vectorizer.get_feature_names_out()[23976])

23976
영화


In [13]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다.
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다.

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다.
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다.

print('예술영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names_out()[w1[i][0]], end=', ')

print('\n')

print('일반영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names_out()[w2[i][0]], end=', ')

예술영화를 대표하는 단어들:
그녀, 자신, 시작, 위해, 사랑, 사람, 영화, 친구, 남자, 가족, 이야기, 마을, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 여자, 대한, 서로, 과연, 시간, 다시, 아들, 소녀, 아내, 다른, 영화제, 사이, 세계, 사실, 하나, 점점, 남편, 감독, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 마지막, 아빠, 생활, 통해, 모습, 기억, 죽음, 비밀, 학교, 음악, 한편, 소년, 생각, 도시, 명의, 결혼, 사고, 전쟁, 위기, 때문, 이제, 최고, 이자, 과거, 일상, 경찰, 간다, 상황, 미국, 운명, 결심, 관계, 현실, 지금, 단편, 여인, 하루, 이름, 이후, 준비, 인간, 만난, 감정, 처음, 국제, 누구, 살인, 충격, 동안, 존재, 그린, 어머니, 연인, 계속, 동생, 작품, 

일반영화를 대표하는 단어들:
자신, 그녀, 영화제, 위해, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 다른, 마을, 시간, 엄마, 아들, 모든, 단편, 마음, 사실, 다시, 세계, 모습, 작품, 통해, 생각, 서로, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 하나, 우리, 애니메이션, 때문, 여성, 죽음, 과연, 점점, 인간, 생활, 한편, 결혼, 상황, 모두, 기억, 명의, 소년, 여행, 가장, 간다, 순간, 이제, 도시, 비밀, 학교, 과거, 가지, 이자, 경찰, 마지막, 미국, 동안, 전쟁, 주인공, 대해, 존재, 현실, 연출, 사고, 살인, 일상, 어머니, 계속, 사회, 인생, 다큐멘터리, 부문, 섹스, 최고, 바로, 의도, 동생, 하루, 위기, 계획, 정체, 한국, 

In [14]:
n = 15
w1_, w2_ = [], []
for i in range(100):
    w1_.append(vectorizer.get_feature_names_out()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names_out()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [15]:
print(target_art)

['아빠', '음악', '운명', '결심', '지금', '여인', '이름', '이후', '준비', '만난', '감정', '처음', '누구', '충격', '그린']


In [16]:
print(target_gen)

['서울', '애니메이션', '여성', '가지', '주인공', '대해', '연출', '사회', '다큐멘터리', '부문', '섹스', '바로', '의도', '계획', '정체']


TF-IDF로 장르 별 대표 단어 추출하기

In [17]:
genre_txt = ['synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_action.txt', 'synopsis_comedy.txt', 'synopsis_war.txt', 'synopsis_horror.txt']
genre_name = ['드라마', '멜로로맨스', '액션', '코미디', '전쟁', '공포(호러)']

print("슝~")

슝~


In [18]:
# 약 10분정도 걸립니다.
genre = []
for file_name in genre_txt:
    genre.append(read_token(file_name))

synopsis_drama.txt 파일을 읽고 있습니다.
synopsis_romance.txt 파일을 읽고 있습니다.
synopsis_action.txt 파일을 읽고 있습니다.
synopsis_comedy.txt 파일을 읽고 있습니다.
synopsis_war.txt 파일을 읽고 있습니다.
synopsis_horror.txt 파일을 읽고 있습니다.


In [19]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names_out()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names_out()[w[i][j][0]])
            print(vectorizer.get_feature_names_out()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

드라마: 그녀, 자신, 시작, 위해, 사랑, 사람, 영화, 친구, 남자, 가족, 이야기, 마을, 사건, 마음, 세상, 
멜로로맨스: 자신, 그녀, 영화제, 위해, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 


단어 셋이 적어서 중복 삭제없이 그대로 사용

## TF-IDF 를 LSA로 압축하기

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
import numpy as np

# art = read_token(art_txt)
# gen = read_token(gen_txt)

#genre_txt = ['synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_action.txt', 'synopsis_comedy.txt', 'synopsis_war.txt', 'synopsis_horror.txt']
#genre_name = ['드라마', '멜로로맨스', '액션', '코미디', '전쟁', '공포(호러)']

# 위에서 토큰화 한 거 가져오기
genre_docs = genre

In [21]:
# 1) 문서 리스트 (art, gen + 장르들)
docs_text = [art, gen] + genre_docs
doc_names = ['art','gen'] + genre_name  

In [22]:
# 2) TF-IDF 피팅(모든 문서 공통 어휘로)
vectorizer= TfidfVectorizer()
X = vectorizer.fit_transform(docs_text)  # shape: (2 + #genres, vocab)
terms = vectorizer.get_feature_names_out()

In [1]:
# 3) LSA(SVD) 적용
k = min(100, max(2, X.shape[1]-1))
svd = TruncatedSVD(n_components=k, random_state=42)
X_lsa_doc = svd.fit_transform(X)   # (docs x k)  각 문서의 LSA 벡터
term_load = svd.components_.T                # (vocab x k) 각 단어의 로딩 벡터
term_load = normalize(term_load)             # 코사인 유사도 계산 유리하게 정규화

NameError: name 'X' is not defined

## TF-IDF로 LSA 기반으로 단어 셋 만들기

In [1]:
# lsa 상위 단어 뽑기 정의
import numpy as np
from sklearn.preprocessing import normalize

def centroid(rows: np.ndarray):
    """rows: (n_docs x k). 문서군 중심벡터 L2 정규화."""
    c = rows.mean(axis=0)
    return c / (np.linalg.norm(c) + 1e-12)

def top_words_lsa(center_vec, term_load, terms, topn=200):
    """center_vec: (k,), term_load: (vocab x k), terms: vocab list/ndarray"""
    c = center_vec / (np.linalg.norm(center_vec) + 1e-12)
    scores = term_load.dot(c)  # (vocab,)
    order  = np.argsort(-scores)
    return [terms[i] for i in order[:topn]]


In [ ]:
#2) 타깃(X=예술, Y=일반) LSA 단어셋 뽑기

# docs_text 순서: [art, gen] + genre_docs 라고 가정
idx_art, idx_gen = 0, 1
cent_art = X_lsa_doc[idx_art]  # 문서 하나면 그대로 벡터
cent_gen = X_lsa_doc[idx_gen]

# 후보군 넉넉히 뽑고 나중에 n개로 줄이기
TOPN_CANDIDATES = 200
X_art_cand = top_words_lsa(cent_art, term_load, terms, topn=TOPN_CANDIDATES)
Y_gen_cand = top_words_lsa(cent_gen, term_load, terms, topn=TOPN_CANDIDATES)

In [ ]:
# 3) 장르별 속성(A_g, B_g) LSA 단어셋 뽑기

num_genres = len(docs_text) - 2
idx_genres = list(range(2, 2 + num_genres))

def genre_attr_sets_lsa(gi, X_lsa_doc, term_load, terms, topn=200):
    c_g   = X_lsa_doc[gi]               # 장르 g 문서(한 개) 벡터
    others = [j for j in idx_genres if j != gi]
    c_not = centroid(X_lsa_doc[others]) # 나머지 장르들의 중심
    A_g = top_words_lsa(c_g,   term_load, terms, topn=topn)
    B_g = top_words_lsa(c_not, term_load, terms, topn=topn)
    return A_g, B_g

In [ ]:
# LSA 후보를 최종 타깃 단어셋(ART, GEN)으로 정

def pick_unique_in_vocab(primary_list, other_list, n, vocab):
    out = []
    for w in primary_list:
        if w in vocab and w not in other_list and w not in out:
            out.append(w)
            if len(out) == n: break
    if len(out) < n:  # 모자라면 겹치더라도 채움
        for w in primary_list:
            if w in vocab and w not in out:
                out.append(w)
                if len(out) == n: break
    return out

FINAL_N = 15  # 최종 사용 단어 수
target_art = pick_unique_in_vocab(X_art_cand, Y_gen_cand, FINAL_N, embed_vocab)
target_gen = pick_unique_in_vocab(Y_gen_cand, X_art_cand, FINAL_N, embed_vocab)

print("Target(art):", target_art)
print("Target(gen):", target_gen)


In [ ]:
# 장르별 Attribute 만들기

TOPN_ATTR = 200  # 후보 폭넓게 뽑고

lsa_attributes = []
for offset, gname in enumerate(genre_name):  # genre_name은 이미 있음
    gi = 2 + offset  # docs_text에서 해당 장르의 문서 인덱스
    A_cand, B_cand = genre_attr_sets_lsa(gi, X_lsa_doc, term_load, terms, topn=TOPN_ATTR)

    # in-vocab & 최종 N개
    A = [w for w in A_cand if w in embed_vocab][:FINAL_N]
    B = [w for w in B_cand if w in embed_vocab][:FINAL_N]

    lsa_attributes.append((gname, A, B))
    # 확인용 출력
    # print(gname, "A:", A)
    # print(gname, "B:", B)


In [ ]:
# WEAT 계산
import numpy as np

def s_word(w, A, B, model):
    a = [model.wv.similarity(w, x) for x in A if x in model.wv]
    b = [model.wv.similarity(w, x) for x in B if x in model.wv]
    if not a or not b:
        return 0.0
    return float(np.mean(a) - np.mean(b))

def weat_score(X, Y, A, B, model):
    sx = np.sum([s_word(x, A, B, model) for x in X if x in model.wv])
    sy = np.sum([s_word(y, A, B, model) for y in Y if y in model.wv])
    return float(sx - sy)

rows = []
for gname, A, B in lsa_attributes:
    score = weat_score(target_art, target_gen, A, B, model)
    rows.append((gname, score, len(A), len(B)))

for g, s, na, nb in rows:
    print(f"{g:>10s} | WEAT(LSA-based) = {s: .4f} | |A|={na}, |B|={nb}")


In [ ]:
import pandas as pd, matplotlib.pyplot as plt
df = pd.DataFrame(rows, columns=["genre","weat","|A|","|B|"]).set_index("genre")
print(df.round(4))

plt.figure(figsize=(3.6, 0.45*len(df)+1))
plt.imshow(df[["weat"]].values, aspect='auto')
plt.colorbar(label="WEAT (X=art, Y=gen | A=genre, B=~genre)")
plt.yticks(range(len(df.index)), df.index)
plt.xticks([0], ["Score"])
plt.title("Genre-wise WEAT (LSA-based)")
plt.tight_layout()
plt.show()
